# Лабораторна робота 3 з "Асиметричних криптосистем та протоколів"
## Тема: Криптосистема Рабіна; Атака на протокол доведення знання без розголошення

**Виконали**\
Дигас Богдан, ФІ-03\
Починок Юрій, ФІ-03